In [24]:
from __future__ import division
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import numbeo_scraper as ns
import get_bea_data as gbd
import population_cleanup as pc
%matplotlib inline

In [42]:
# read in population (1790 - 2010) and rj metrics meetup info (2013-2014) and merge df's

pop_df = pc.get_pop_data('data/1790-2010_MASTER.csv')
rj_df = pc.get_rj_data('data/rj_metrics.txt')
new_df = pd.concat([pop_df, rj_df], axis=1)
new_df.shape

(108, 40)

In [45]:
# clean and join bureau of economic affairs info

raw_bea = gbd.get_bea_data('http://www.bea.gov/newsreleases/regional/gdp_metro/2015/xls/gdp_metro0915.xls')
bea_df = gbd.clean_me(raw_bea)
next_df = pd.concat([new_df, bea_df], axis=0)
next_df.shape

(492, 48)

In [47]:
url = 'https://www.biggestuscities.com/demographics/us/education-college-graduates-by-top-100-city'
# This works, don't break!'

def get_grad_data(url, cols, skip_rows=4):
    file_name = url.split('/')[-1].replace('-', '_')
    path = os.getcwd()+'/data/biggestuscities' #/{}'.format(file_name)
    file_path = '{}/{}.csv'.format(path, file_name)
    
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.isfile(file_path):
        soup = ns.get_pages(url)
        tabs =[]
        table = soup[0].findAll('table')

        for tag in table:
            tabs.append(tag.text)

        d = [t.replace("\n",",").strip() for t in tabs][-1]
        d = d.split(' ')
        t = [item for item in d if item != '']
        t_done = [item.replace(',',' ').strip()  for item in t]
        just_table = t_done[skip_rows:-9]
        bad_item_list = ['(adsbygoogle', '=', 'window.adsbygoogle', '||', '[]).push({', '});', 'San',
                        'District', 'Of', 'North', 'St.', 'City', 'New', 'Springs', 'Urban', 'Beach',
                         'Rouge', 'Los', 'El', 'Vista', 'County', 'Fort', 'Las', 'Christi', 'Ana']

        for item in bad_item_list:
            just_table.remove(item)

        just_table = [item for item in just_table if item not in bad_item_list if item]
        just_table = [item.replace('Francisco', 'san_francisco') for item in just_table]
        just_table = [item.replace('Carolina', 'north_carolina') for item in just_table]
        just_table = [item.replace('Baton', 'baton_rouge') for item in just_table]
        just_table = [item.replace('Chula', 'chula_vista') for item in just_table]
        just_table = [item.replace('Worth', 'fort_worth') for item in just_table]
        just_table = [item.replace('Christi', 'corpus_christi') for item in just_table]
        just_table = [item.replace('Santa', 'santa_ana') for item in just_table]
        just_table = [item.replace('Vegas', 'las_vegas') for item in just_table]
        just_table = [item.replace('Bernadino', 'san_bernadino') for item in just_table]
        just_table = [item.replace('Columbia', 'dc') for item in just_table]
        just_table = [item.replace('Diego', 'san_diego') for item in just_table]
        just_table = [item.replace('Jersey', 'new_jersey') for item in just_table]
        just_table = [item.replace('Orleans', 'new_orleans') for item in just_table]
        just_table = [item.replace('York', 'new_york') for item in just_table]
        just_table = [item.lower() for item in just_table]

        chunks = [just_table[x:x+4] for x in xrange(0, len(just_table), 4)]
        df = pd.DataFrame(chunks)
        df.columns = cols
        file_name = url.split('/')[-1].replace('-', '_')
        df=df.drop('rank', axis=1)
        df.set_index('city', inplace=True)
        df_csv = df.copy()
        df_csv.to_csv(file_path)
        return df
    else:
        df = pd.read_csv(file_path)
        df.set_index('city', inplace=True)
        return df

cols = ['rank', 'city','state_bc','pct_college_grads_bc']
df = get_grad_data(url, cols)
another_df = pd.concat([next_df, df], axis=0)

#df.to_csv('/data/biggestuscities/{}'.format(filename))
another_df.shape

(592, 50)

In [263]:
url = 'https://www.biggestuscities.com/demographics/us/education-college-graduates-by-top-100-city'
cols = ['rank', 'city','state_fb','pct_foreign_born']
df2 = get_grad_data(url, cols, 6)

'/Users/IXChris/Desktop/G/capstone/data/biggestuscities/people_foreign_born_by_top_100_city.csv'

In [230]:
#subset by dense rows
meetup_df = new_df[new_df['Pop'].notnull()]
cities = list(meetup_df.index)

# fix column names
cols = meetup_df.columns
cols = [item.lower().replace(' ', '_') for item in cols]